<a href="https://colab.research.google.com/github/aldiekawahyu/manajerku-inventory-management/blob/main/eda_demand_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download data from https://www.kaggle.com/felixzhao/productdemandforecasting
Steps: \\
1. Go to kaggle account and click **'Create New API Token'**. A kaggle.json file will be downloaded to your local machine.
1. Upload the kaggle.json file into into colab
1. Download the dataset from kaggle and unzip it

In [12]:
import os

! pip install -q kaggle
from google.colab import files
files.upload()
try:
  os.mkdir('~/.kaggle')    
except:
  pass
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d felixzhao/productdemandforecasting/
! unzip productdemandforecasting.zip

Saving kaggle.json to kaggle (2).json
productdemandforecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  productdemandforecasting.zip
replace Historical Product Demand.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Historical Product Demand.csv  


# Start Exploratory Data Analysis

## Check Missing Value and Object Type

First, open the csv file as pandas dataframe file then check if there are missing values.

In [13]:
import pandas as pd

df = pd.read_csv('Historical Product Demand.csv')
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


Sum of missing value per column.

In [14]:
df.isnull().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

Proportion of missing value per column.

In [15]:
df.isnull().sum()/len(df)

Product_Code        0.000000
Warehouse           0.000000
Product_Category    0.000000
Date                0.010718
Order_Demand        0.000000
dtype: float64

It can be seen that there are missing values in *Date* column. We are going to handle this by **dropping rows containing missing value**, because date is non-numerical value thus it cannot be imputed.

In [26]:
print('Length before dropping: {}'.format(len(df)))
df = df.dropna(axis='rows')
print('Length after dropping: {}'.format(len(df)))

Length before dropping: 1048575
Length after dropping: 1037336


Next, we will check the type of each column of the data frame and convert the type of *Order_Demand* column if it is non-numeric.

In [18]:
df.dtypes

Product_Code        object
Warehouse           object
Product_Category    object
Date                object
Order_Demand        object
dtype: object

We see that *Order_Demand* column is not a float, so there must be something about the number. Let's try to convert it into float.

In [19]:
try:
  df["Order_Demand"] = pd.to_numeric(df["Order_Demand"], downcast="float")
  print('Conversion succeeded')
except ValueError as e:
  print('Cannot do conversion: ',e)

Cannot do conversion:  Unable to parse string "(1)" at position 112290


We see an indication that several numbers in *Order_Demand* column are written in parentheses, so we need to remove the parentheses.

In [20]:
def remove_parentheses(x):
  import re
  return re.sub('[()]', '', x)

df["Order_Demand"] = df["Order_Demand"].apply(remove_parentheses)

Lets try to do conversion again.

In [23]:
try:
  df["Order_Demand"] = pd.to_numeric(df["Order_Demand"], downcast="float")
  print('Conversion succeeded')
except ValueError as e:
  print('Cannot do conversion: ',e)

Conversion succeeded


In [24]:
df.dtypes

Product_Code         object
Warehouse            object
Product_Category     object
Date                 object
Order_Demand        float32
dtype: object

The *Order_Demand* column is already converted to float.

## 